<a href="https://colab.research.google.com/github/yousef-taheri/Projects/blob/master/NeuralNetworks/NeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.models import Sequential
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [ ]:
Golub_X = pd.read_csv('data/Golub_X',sep=' ',header=None) # Observations
Golub_y = pd.read_csv('data/Golub_y',sep=' ',header=None) # Classes
Golub_X=Golub_X.values
Golub_y=Golub_y.values.squeeze()

X = pd.read_csv('data/Breast.txt',sep=' ')
Breast_y = X.values[:,30] # Classes
Breast_y=Breast_y==1
Breast_X = X.values[:,0:29] # Observations



In [ ]:
#read splex dataset
SPLEX_env=pd.read_csv('data/SPLEX_env.txt',sep=' ')
SPLEX_host=pd.read_csv('data/SPLEX_host.txt',sep=' ')
SPLEX_micro=pd.read_csv('data/SPLEX_micro.txt',sep=' ')

SPLEX_class=pd.read_csv('data/classes.csv',sep=',')

#remove rows where SPLEX class in NA
non_NA_indexs=~SPLEX_class.isna().squeeze().values

SPLEX_env=SPLEX_env.iloc[non_NA_indexs,:]
SPLEX_host=SPLEX_host.iloc[non_NA_indexs,:]
SPLEX_micro=SPLEX_micro.iloc[non_NA_indexs,:]
SPLEX_class=SPLEX_class.iloc[non_NA_indexs,:]

SPLEX=pd.concat([SPLEX_env,SPLEX_host,SPLEX_micro],axis=1)

In [ ]:
sk_model = MLPClassifier(solver='lbfgs', alpha=1e-7,hidden_layer_sizes=(10, 5))
#sk_model.fit(XTrain, yTrain)
#yTest_predicted =sk_model.predict(XTest)

In [ ]:
from sklearn.model_selection import cross_val_score
print('10 fold accuracy score for Gloub data',np.mean(cross_val_score(sk_model,Golub_X, Golub_y, cv=10)))
print('10 fold accuracy score for Breast cancer data',np.mean(cross_val_score(sk_model,Breast_X, Breast_y, cv=10)))

10 fold accuracy score for Gloub data 0.9571428571428573
10 fold accuracy score for Breast cancer data 0.9612468671679197


In [ ]:
#keras MLP classifier with cv score calculation
def keras_cross_val_score(X,y):
  in_dim=X.shape[1]
  model = Sequential()
  model.add(Dense(32, activation='relu',input_dim=in_dim))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

  batch_size=10
  epochs=100

  skf=StratifiedKFold(n_splits=10)

  scores=[]
  for train,test in skf.split(X,y):
    model.fit(X[train], y[train],batch_size=batch_size,verbose=0,epochs=epochs,validation_split=0.1)
    score = model.evaluate(X[test], y[test],batch_size=batch_size, verbose=0)
    scores.append(score[1])
  return scores


In [ ]:
keras_cv_score_Breast=np.mean(keras_cross_val_score(Breast_X, Breast_y))
print("keras model 10 fold cv score",keras_cv_score_Breast)

keras model 10 fold cv score 0.9911654174327851


In [ ]:
keras_cv_score_Breast=np.mean(keras_cross_val_score(Golub_X, Golub_y))
print("keras model 10 fold cv score",keras_cv_score_Breast)

keras model 10 fold cv score 0.9589285731315613


In [ ]:
Gene_count_y=(SPLEX_class=='HGC').squeeze().values

In [ ]:
keras_cv_score_SPLEX_env=np.mean(keras_cross_val_score(SPLEX_env.values, Gene_count_y))
print("keras model 10 fold cv score",keras_cv_score_SPLEX_env)

keras model 10 fold cv score 0.8050000011920929


In [ ]:
keras_cv_score_SPLEX_host=np.mean(keras_cross_val_score(SPLEX_host.values, Gene_count_y))
print("keras model 10 fold cv score",keras_cv_score_SPLEX_host)

keras model 10 fold cv score 0.8700000017881393


In [ ]:
keras_cv_score_SPLEX_micro=np.mean(keras_cross_val_score(SPLEX_micro.values, Gene_count_y))
print("keras model 10 fold cv score",keras_cv_score_SPLEX_micro)

keras model 10 fold cv score 0.9150000035762786


In [ ]:
keras_cv_score_SPLEX=np.mean(keras_cross_val_score(SPLEX.values, Gene_count_y))
print("keras model 10 fold cv score",keras_cv_score_SPLEX)

keras model 10 fold cv score 0.8850000023841857
